In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
class GemmaRouter:
    """
    A simple wrapper for using Google's Gemma-2-2B-IT model
    via Hugging Face's OpenAI-compatible inference API.
    """

    def __init__(
        self,
        model: str = "google/gemma-2-2b-it:nebius",
        token_env: str = "HF_THIRD_TOKEN",
        base_url: str = "https://router.huggingface.co/v1",
    ):
        load_dotenv()
        api_key = os.getenv(token_env)
        if not api_key:
            raise ValueError(
                f"Missing token: environment variable '{token_env}' not found."
            )

        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.model = model

    def invoke(self, prompt: str, temperature: float = 0.7, max_tokens: int = 256):
        """Non-streaming response."""
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return completion.choices[0].message.content.strip()

    def stream_invoke(self, prompt: str, temperature: float = 0.7, max_tokens: int = 256):
        """Streaming response."""
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
            stream=True,
        )
        return stream
model2=GemmaRouter()


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [3]:
model.invoke("what is the main cause that you think that Itachi Uhciha had to commit such a crime ")

AIMessage(content='The main cause that Itachi Uchiha had to commit the Uchiha Clan Massacre was a **tragic and impossible choice between the greater good of Konoha and the survival of his clan**, driven by his deep love for his village and, crucially, his profound desire to protect his younger brother, Sasuke.\n\nHere\'s a breakdown of the complex factors:\n\n1.  **The Uchiha Coup d\'état:** The Uchiha clan, feeling marginalized and discriminated against by Konoha\'s leadership (especially after being blamed for the Nine-Tails attack), was planning a full-scale coup to overthrow the village government. This would have inevitably led to a devastating civil war within Konoha.\n\n2.  **Itachi\'s Loyalty to Konoha and Desire for Peace:** Itachi was a child prodigy who had witnessed the horrors of the Third Great Ninja War at a very young age. This experience instilled in him a profound desire for peace and a deep loyalty to Konoha as the entity that maintained that peace. He understood tha

In [7]:
from pydantic import BaseModel, Field
from typing import Optional
class Person(BaseModel):
    """Information about a person."""

    name: Optional[str] = Field(default=None, description="The name of the person")
    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )
    country: Optional[str] = Field(
        default=None, description="The country of the person if known"
    )

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)


In [11]:
chain = prompt | model.with_structured_output(schema=Person)

In [12]:

comment = "I absolutely love this product! It's been a game-changer for my daily routine. The quality is top-notch and the customer service is outstanding. I've recommended it to all my friends and family. - Sarah Johnson, USA"

response = chain.invoke({"text": comment})


In [13]:
print(response)

name='Sarah' lastname='Johnson' country='USA'
